In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns

# 1. Constraints analysis

### 1.1 Example setup

In [ ]:
from angle_set import AngleSet

d = 2 # do not change.
N = 5

np.random.seed(51)

angle_set = AngleSet(N=N, d=d)
angle_set.set_points(mode='random')
angle_set.plot_all()

In [ ]:
Apoly, bpoly = angle_set.get_polygon_constraints([3])
plt.matshow(Apoly)

In [ ]:
angle_set.plot_all()
points = angle_set.points
Arays, brays = angle_set.get_ray_constraints(verbose=False)
print(Arays.shape)
print(np.linalg.matrix_rank(Arays))

In [ ]:
plt.matshow(np.hstack((Arays, brays.reshape((-1, 1))))); plt.colorbar()
assert np.allclose(Arays.dot(angle_set.theta), brays.flatten())
print(angle_set.N)

In [ ]:
from angle_set import combine_fullrank_matrix

Afull, bfull = combine_fullrank_matrix(Arays, Apoly, brays, bpoly, False)
print('kept rows: {} of {}'.format(Afull.shape[0] - Arays.shape[0], Apoly.shape[0]))
plt.matshow(np.c_[Afull, bfull])

In [ ]:
def get_numbers(N):
    import scipy
    sum_ = 0
    for i in range(1, angle_set.N-2): # goes to N-3
        sum_ += i
    n_rays = angle_set.N * sum_
    n_poly = int(scipy.special.binom(angle_set.N-1, 2))
    return n_rays, n_poly

n_rays, n_poly = get_numbers(angle_set.N)
assert n_rays == Arays.shape[0]
print('number of ray constraints found:', n_rays)

In [ ]:
print('number of poly constraints found:', n_poly)
assert n_poly == Afull.shape[0] - Arays.shape[0]

### 1.2 Compare obtained number of constraints

In [ ]:
from helpers import savefig
import seaborn as sns

def plot_here(ax, range_):
    colors = sns.color_palette('tab20')
    ax.stackplot(Ns[range_], 
                 [n_rays_list[range_], n_poly_list[range_], n_nonlinear_list[range_], DOFs[range_]], 
                 labels=['single', 'triangle', 'non-linear', 'DOF'], 
                 colors=colors)
    ax.plot(Ns[range_], Ms[range_], 
            color='black', label='no. angles $M$', ls=':')
    ax.set_xlim(Ns[range_][0], Ns[range_][-1])
    
def plot_inset(ax, loc=[0.5, 0.5, 0.47, 0.47], range_full=[0, 10],
               range_inset = [14, 15, 1000, 1500]):
    """ Create zoom inside the plot. """
    axins = ax.inset_axes(loc) # location (x,y), size
    plot_here(axins, range_full)
    # sub region of the original image
    x1, x2, y1, y2 = range_inset
    axins.set_xlim(x1, x2)
    axins.set_ylim(y1, y2)
    axins.set_xticklabels('')
    axins.set_yticklabels('')
    axins.set_xticks([])
    axins.set_yticks([])
    axins.set_facecolor((1, 1, 1, 0.5))
    r_, lines_ = ax.indicate_inset_zoom(axins, label=None)
    [l.set_color('black') for l in lines_]
    [l.set_alpha(1.0) for l in lines_]
    r_.set_alpha(1.0)
    r_.set_edgecolor('black')

In [ ]:
d = 2 # do not change.

print('N \t M \t DOF \t necessary \t linear \t nonlinear')

Ns = np.arange(4, 20)
Ms = np.empty((len(Ns),))
DOFs = np.empty((len(Ns),))
n_linear_list = np.empty((len(Ns),))
n_nonlinear_list = np.empty((len(Ns),))
n_poly_list = np.empty((len(Ns),))
n_rays_list = np.empty((len(Ns),))

for i, N in enumerate(Ns):
    angle_set = AngleSet(N=N, d=d)
    angle_set.set_points(mode='random')
    n_rays = angle_set.get_n_rays() 
    n_poly = angle_set.get_n_poly()

    Apoly, bpoly = angle_set.get_polygon_constraints([3])
    Aray, bray = angle_set.get_ray_constraints()
    
    # Sanity check only. is taking too long for N >= 11
    if N < 11:
        Afull, bfull = combine_fullrank_matrix(Aray, Apoly, bray.flatten(), bpoly, False)
        assert n_rays == Aray.shape[0]
        assert n_poly == Afull.shape[0] - Aray.shape[0]
    
    Afull = np.vstack([Aray, Apoly[:n_poly]])
    bfull = np.hstack([bray, bpoly[:n_poly]])
    
    #DOF = angle_set.N * angle_set.d - 2*angle_set.d - 1
    DOF = angle_set.get_DOF()
    necessary = angle_set.M - DOF
    
    nonlinear = sum([1] + [i for i in range(2, angle_set.N-2)])
    assert nonlinear == angle_set.get_n_sine()
    
    assert nonlinear + n_rays + n_poly == necessary
    print('{} \t {} \t {} \t {} \t {} \t {}'.format(
        angle_set.N, angle_set.M,  DOF, necessary, n_rays + n_poly, nonlinear
    ))
    
    n_poly_list[i] = n_poly
    n_rays_list[i] = n_rays
    n_linear_list[i] = n_rays + n_poly
    n_nonlinear_list[i] = nonlinear
    Ms[i] = angle_set.M
    DOFs[i] = DOF
                       
print('looks like things are working.')

### 1.3 Create plot for paper

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(6, 2.5)
range_ = np.arange(14)
plot_here(ax, range_)
plot_inset(ax, 
           loc=[0.02, 0.25, 0.5, 0.35], 
           range_full=range_,
           range_inset=[4, 8, 0, 200])
h, l = ax.get_legend_handles_labels()
ax.legend(h[::-1], l[::-1], loc='upper left',ncol=3)
#ax.legend(loc='lower right',ncol=1)
ax.set_ylabel('count', fontsize=12)
ax.set_xlabel('number of points $N$', fontsize=12)
plt.tight_layout()
ax.grid()

savefig(fname='plots/number_constraints.pdf')

# 2. Do reconstruction and check discrepancy.

### 2.1 Check that everything works on an example setup

In [ ]:
from angle_set import AngleSet
from pylocus.algorithms import procrustes
from algorithms import reconstruct_from_angles

d = 2 
N = 5
np.random.seed(51)
angle_set = AngleSet(N=N, d=d)
angle_set.set_points(mode='random')
angle_set.plot_all()

points = reconstruct_from_angles(angle_set.theta_tensor)
points_fitted, *_ = procrustes(angle_set.points, points, scale=True)
points_old = angle_set.reconstruct_aloc(angle_set.theta)

plt.figure()
plt.scatter(*points_fitted.T, label='fitted')
plt.scatter(*points_old.points.T, label='fitted old')
plt.scatter(*angle_set.points.T, label='original', marker='x')
plt.axis('equal')
plt.legend(loc='best')

In [ ]:
def get_linear_constraints(angle_set, full_rank=True):
    n_rays, n_poly = get_numbers(angle_set.N)
    Apoly, bpoly = angle_set.get_polygon_constraints([3])
    Aray, bray = angle_set.get_ray_constraints()
    if full_rank:
        Afull = np.vstack([Aray, Apoly[:n_poly]])
        bfull = np.hstack([bray, bpoly[:n_poly]])
    else:
        Afull = np.vstack([Aray, Apoly])
        bfull = np.hstack([bray, bpoly])
    return Afull, bfull

In [ ]:
from pylocus.basics_angles import get_theta_tensor
from pylocus.basics import projection

# create noisy angle vector
def get_noisy(vec):
    theta_noisy = vec.copy()
    theta_noisy += np.random.normal(scale=0.1, loc=0, size=theta_noisy.shape)
    return theta_noisy

theta_noisy = get_noisy(angle_set.theta)

# reconstruct raw
theta_noisy_tensor = get_theta_tensor(theta_noisy, angle_set.corners, angle_set.N)

points_noisy_unfit = reconstruct_from_angles(theta_noisy_tensor)
points_noisy, *_ = procrustes(angle_set.points, points_noisy_unfit, scale=True)

# impose linear constraints
Afull, bfull = get_linear_constraints(angle_set)
bfull = bfull.flatten()
theta_linear, __, __ = projection(theta_noisy, Afull, bfull)
theta_linear_tensor = get_theta_tensor(theta_linear, angle_set.corners, angle_set.N)

points_linear_unfit = reconstruct_from_angles(theta_linear_tensor)
points_linear, *_ = procrustes(angle_set.points, points_linear_unfit, scale=True)

# plot results
plt.figure()
plt.scatter(*points_linear.T, label='linear')
plt.scatter(*points_noisy.T, label='noisy')
plt.scatter(*angle_set.points.T, label='original', marker='x')
plt.axis('equal')
plt.legend(loc='best')

### 2.2 Find out which constraints imply other constraints.

In [ ]:
from algorithms import solve_constrained_optimization, constraint_sine_multi

# choose which sine constraints to include.
if N==4:
    choices_sine = range(1)
elif N==5: 
    choices_sine = range(3)
elif N==6: 
    choices_sine = range(6) # range(7) 

print('minimized:')
theta_sine, success = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                            Afull=Afull,
                                            bfull=bfull, N=N,
                                            choices_sine=choices_sine)
[print(constraint_sine_multi(theta_sine, angle_set.corners, N, [choice])) for choice in choices_sine]

print('not minimized:')
n_available = angle_set.get_n_sine()
other = list(range(n_available))
[other.remove(choice) for choice in choices_sine]
for o in other:
    print(constraint_sine_multi(theta_sine, angle_set.corners, N, [o]))

In [ ]:
d=2
np.random.seed(1)
for N in range(4, 8):
    print('===== N={} ====='.format(N))
    angle_set = AngleSet(N=N, d=d)
    angle_set.set_points('random')
    Atri, btri = angle_set.get_triangle_constraints()
    Aray, bray = angle_set.get_ray_constraints()
    Alinear = np.vstack((Aray, Atri))
    blinear = np.hstack((bray, btri))
    print('considering only the minimal constraints:')
    print('shape:', Alinear.shape, 'rank:', np.linalg.matrix_rank(Alinear))

    Apoly, bpoly = angle_set.get_polygon_constraints(range(3, N))
    Alinear = np.vstack((Aray, Apoly))
    blinear = np.hstack((bray, bpoly))
    print('considering more constraints:')
    print('shape:', Alinear.shape, 'rank:', np.linalg.matrix_rank(Alinear))

### 2.3 Calculate discrepancy

In [ ]:
from angle_set import create_theta
from simulation import mae
from algorithms import reconstruct_theta

N=4
d=2
np.random.seed(1)
angle_set = AngleSet(N=N, d=d)
angle_set.set_points('random')


num_sine = angle_set.get_n_sine()
num_linear = angle_set.get_n_linear()
choices_sine = range(num_sine)
choices_linear = range(num_linear)

# denoise
theta_noisy = get_noisy(angle_set.theta)
Afull, bfull = get_linear_constraints(angle_set)

theta_sine, __ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                            Afull=Afull, bfull=bfull, N=N,
                                            choices_sine=choices_sine, 
                                            choices_linear=choices_linear)

# reconstruct
theta_sine_reconstructed, points_sine = reconstruct_theta(theta_sine, angle_set.corners, angle_set.N)

mae_noisy = mae(angle_set.theta, theta_noisy)
mae_denoised = mae(angle_set.theta, theta_sine)
mae_discrepancy = mae(theta_sine_reconstructed, theta_sine)
print(' noisy:       {:2.2e}\n denoised:    {:2.2e}\n discrepancy: {:2.2e}\n'.format(mae_noisy, mae_denoised, mae_discrepancy))

### 2.4 Test discrepancy for increasing number of constraints

In [ ]:
def plot_discrepancy(thetas_sine, thetas_sine_reconstructed,
                     thresh=None, labels=['linear constraints', 'sine constraints']):
    fig, ax = plt.subplots()
    fig.set_size_inches(5, 2.5)
    label = labels[0]
    for n_total in thetas_sine.keys():
        theta_sine = thetas_sine[n_total]
        theta_sine_reconstructed = thetas_sine_reconstructed[n_total]

        mae_noisy = mae(angle_set.theta, theta_noisy)
        mae_denoised = mae(angle_set.theta, theta_sine)
        mae_discrepancy = mae(theta_sine_reconstructed, theta_sine)
        #print(' noisy:{:2.2e} denoised: {:2.2e} discrepancy: {:2.2e}'.format(mae_noisy, mae_denoised, mae_discrepancy))
        if n_total == 0:
            ax.scatter(n_total, mae_discrepancy, color='black')
        elif (thresh is None) or (n_total <= thresh):
            ax.scatter(n_total, mae_discrepancy, color='C0', label=label)
            label = None
        else:
            ax.scatter(n_total, mae_discrepancy, color='C1', label=label)
            label = None
        if n_total == thresh: 
            label = labels[1]
    ax.axvline(n_linear + n_sine, color='black', label='bound')
    #ax.legend(loc='upper right')
    ax.legend()
    #ax.set_ylim(-0.1, 0.7)
    ax.grid()
    ax.set_xlabel('number of constraints')
    ax.set_ylabel('angle discrepancy')
    plt.tight_layout()
    return fig, ax

### first linear then sine constraints

In [ ]:
# required number
from math import floor
from simulation import generate_linear_constraints

Alearn, blearn = generate_linear_constraints(angle_set.points)
print(Alearn.shape)
print(Afull.shape)
print(angle_set.get_n_linear())

Amore, bmore = get_linear_constraints(angle_set, full_rank=False)

A = Afull
b = bfull

num_sine = angle_set.get_n_sine()
all_sine  = int(floor(num_sine * 1.5))
all_linear = A.shape[0] #angle_set.get_n_linear()

thetas_noisy_0 = {}
thetas_sine_0 = {}
thetas_sine_reconstructed_0 = {}
thetas_learned_0 = {}
thetas_learned_reconstructed_0 = {}
for n_linear in range(all_linear + 1):
    if n_linear < all_linear:
        num_sine = 0
    else:
        num_sine = all_sine
        
    for n_sine in range(num_sine + 1):
        if n_sine > 0:
            print('n_sine {}/{}'.format(n_sine, num_sine))
        choices_sine = range(n_sine)
        
        n_total = n_linear + n_sine
        print('n_total', n_total)

        choices_linear = range(n_linear)
        
        # analytic constraints
        theta_sine, __ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                                        Afull=A, bfull=b, N=N,
                                                    choices_sine=choices_sine, 
                                                    choices_linear=choices_linear, 
                                                    eps=None)
        theta_sine_reconstructed, __ = reconstruct_theta(theta_sine, angle_set.corners, angle_set.N)
        
        thetas_noisy_0[n_total] = theta_noisy
        thetas_sine_0[n_total] = theta_sine
        thetas_sine_reconstructed_0[n_total] = theta_sine_reconstructed
        
        # learned constraints
        if n_linear <= Alearn.shape[0]:
            theta_learned, __ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                                            Afull=Alearn, bfull=blearn, N=N,
                                                        choices_sine=choices_sine, 
                                                        choices_linear=choices_linear)
            theta_learned_reconstructed, __ = reconstruct_theta(theta_learned, angle_set.corners, angle_set.N)
            thetas_learned_0[n_total] = theta_learned
            thetas_learned_reconstructed_0[n_total] = theta_learned_reconstructed

In [ ]:
fig, ax = plot_discrepancy(thetas_sine_0, thetas_sine_reconstructed_0, 
                           thresh=all_linear)
ax.set_title('analytical')

fig, ax = plot_discrepancy(thetas_learned_0, thetas_learned_reconstructed_0, 
                           thresh=all_linear)
ax.set_title('learned')
#savefig('plots/discrepancy.pdf', fig=fig)

### first sine constraints, then linear

In [ ]:
num_sine = angle_set.get_n_sine() # we don't want to slow down too much. 
print(num_sine)
num_all_linear = angle_set.get_n_linear()

Alearn, blearn = generate_linear_constraints(angle_set.points)

thetas_sine_1 = {}
thetas_sine_reconstructed_1 = {}

thetas_learned_1 = {}
thetas_learned_reconstructed_1 = {}

for n_sine in range(num_sine+1):
    print('n_sine', n_sine)
    choices_sine = range(n_sine)
    
    if n_sine < num_sine:
        num_linear = 0
    else:
        num_linear = num_all_linear 
        
    for n_linear in range(num_linear+1):
        if n_linear > 0:
            print('n_linear {}/{}'.format(n_linear, num_linear))
            print('n_total', n_total)
        choices_linear = range(n_linear)
        
        n_total = n_linear + n_sine

        theta_sine,__ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                                    Afull=Afull, bfull=bfull, N=N,
                                                    choices_sine=choices_sine, 
                                                    choices_linear=choices_linear)
        theta_sine_reconstructed,__ = reconstruct_theta(theta_sine, angle_set.corners, angle_set.N)
        
        
        theta_learn,__ = solve_constrained_optimization(theta_noisy, angle_set.corners,
                                                    Afull=Alearn, bfull=blearn, N=N,
                                                    choices_sine=choices_sine, 
                                                    choices_linear=choices_linear)
        theta_learn_reconstructed,__ = reconstruct_theta(theta_learn, angle_set.corners, angle_set.N)

        thetas_sine_1[n_total] = theta_sine
        thetas_sine_reconstructed_1[n_total] = theta_sine_reconstructed
        thetas_learned_1[n_total] = theta_learn
        thetas_learned_reconstructed_1[n_total] = theta_learn_reconstructed

In [ ]:
fig, ax = plot_discrepancy(thetas_sine_1, thetas_sine_reconstructed_1, 
                           thresh=num_sine, 
                           labels=['sine constraints', 'linear constraints'])
fig, ax = plot_discrepancy(thetas_learned_1, thetas_learned_reconstructed_1, 
                           thresh=num_sine, 
                           labels=['sine constraints', 'linear constraints'])
#savefig('plots/discrepancy_sine_first.pdf', fig)

### 2.5 Create paper for paper

Below results are obtained with simulation.py

In [ ]:
fname1 = 'discrepancy_first'# few iterations
fname2 = 'discrepancy_second'# 50it, too few sine.
fname3 = 'discrepancy_third'# crashed for n=6
fname4 = 'discrepancy_fourth' # until N=8

figsize = (3, 4)

# after fixing some issues
#fname5 = 'discrepancy_fifth'
fname5 = 'discrepancy_learned0' # N=6 solved until there was a weird error.
fname6 = 'discrepancy_learned1' # N=8 solved until i=12

fname7 = 'discrepancy_new0' # potentially old results
fname8 = 'discrepancy_new1' # new results, non-learning

fname9 = 'discrepancy_learned_new1' # newest results for learning, before interruption at 8, i=14
fname10 = 'discrepancy_learned_new0' # newest results for learning, after interruption at 8, i=14


#fnames = [fname7]
#ylim=[1e-4, 2e-1]
#plotname = 'discrepancy'

#fnames = [fname5, fname6]
#ylim=[1e-2, 2]
#plotname = 'discrepancy_learned'

#fnames = [fname8]
#ylim=[1e-4, 2e-1]
#ylim_err=[1e-9, 2e-3]
#plotname = 'discrepancy_new'

fnames = [fname9, fname10]
ylim=[1e-4, 2e-1]
ylim_err=[1e-9, 2e-3]
plotname = 'discrepancy_learned_new'

dfs = [pd.read_pickle('results/{}.pkl'.format(fname)) for fname in fnames]
df = pd.concat(dfs, ignore_index=True, sort=False)

def mae(a, b):
    assert len(a.flatten()) == len(b.flatten())
    return np.sum(np.abs(a.flatten() - b.flatten())) / len(a.flatten())

df.loc[:, 'discrepancy_sine'] = df.apply(lambda row: mae(row.theta_sine, row.theta_sine_reconstructed), axis=1)
df.loc[:, 'discrepancy_noisy'] = df.apply(lambda row: mae(row.theta_noisy, row.theta_noisy_reconstructed), axis=1)
df.loc[:, 'sine'] = df.apply(lambda row: row.n_sine>0, axis=1)
df.tail()

In [ ]:
# plot results
from scipy.special import binom
from helpers import savefig

fig, axs = plt.subplots(2, 1, sharex=True, sharey=False)
fig.set_size_inches(*figsize)
i = 0
Ns = sorted(df.N.unique())

kwargs = dict(
    linewidth=0.0,
    s=20
)
sns.set_palette(sns.color_palette('Blues_d', n_colors=len(Ns)))
legend=False
log=True

label = 0 
for N, df_N in df.groupby('N'):
    N = int(N)
    df_mean = df_N.groupby('n_total').aggregate(np.mean, axis=0)
    df_mean.reset_index(inplace=True, drop=False)
    
    discrepancy_noise=df_mean.discrepancy_noisy.values[0]
    error_noise=df_mean.error_noisy.values[0]
    
    ################ DISCREPANCY PLOT ####################
    ax = axs[0]
    label = 'N = {}'.format(N)
    color = 'C{}'.format(i)
    ax.set_yscale('log')
    sns.scatterplot(data=df_mean, x='n_total', y='discrepancy_sine', ax=ax, 
                    label=label, style='sine', color=color, **kwargs)
    ax.set_xlabel('number of constraints')
    ax.set_ylabel('discrepancy')
    if log:
        ax.set_yscale('log')
        ax.set_ylim(*ylim)
    #ax.axhline(discrepancy_noise, color=color)
    
    ################ ERROR PLOT ####################
    ax = axs[1]
    label = None
    ax.set_yscale('log')
    sns.scatterplot(data=df_mean, x='n_total', y='error_sine', 
                    ax=ax, label=label, style='sine', color=color, **kwargs)
    #ax.axhline(error_noise, color=color)
    if log:
        ax.axis(yscale='log') 
        ax.set_ylim(*ylim_err)
    ax.set_xlabel('number of constraints')
    ax.set_ylabel('accuracy')
    ax.legend().set_visible(False)
    i += 1

################ LEGEND ####################
if legend:
    handles, labels = axs[0].get_legend_handles_labels()
    print(labels)
    labels[1] = 'constraint type'
    labels[2] = 'linear'
    labels[3] = 'non-linear'
    order = range(len(labels))[::4]
    ncol = min(len(order), 3)
    l1 = axs[0].legend([handles[idx] for idx in order],[labels[idx] for idx in order], 
                       loc='center', ncol=ncol, bbox_to_anchor=[0.5, 1.05], 
                       framealpha=1.0)
    l1.remove()
    order = [2,3] # [1, 2, 3]
    l2 = axs[0].legend([handles[idx] for idx in order],[labels[idx] for idx in order],
                        loc='upper center', ncol=2, bbox_to_anchor=[0.5, -0.0],
                        framealpha=1.0)
    l2.remove()
    # to make sure legend is in foreground.
    ax2 = axs[0].twinx()
    ax2.axis('off')
    ax2.add_artist(l2)
    ax2.add_artist(l1)
else:
    l1 = axs[0].get_legend()
    l2 = axs[0].get_legend()
    l1.remove()
    l2.remove()
    
    ys = [10**(-3.8)]*len(Ns)
    ys[0] = 10**(-3.5)
    ys2 = [10**(-8.4)]*len(Ns)
    ys2[0] = 10**(-7.7)
    xs = [-5.0, 14.0, 42.0, 85.0, 130.0]
    if plotname != 'discrepancy_new':
        axs[0].get_yaxis().set_ticklabels([])
        axs[1].get_yaxis().set_ticklabels([])
        axs[0].set_ylabel('')
        axs[1].set_ylabel('')
    xs = [-5.0, 14.0, 42.0, 85.0, 130.0]
    for j, N in enumerate(Ns):
        axs[0].annotate(s='N={}'.format(int(N)), 
                       xy=(xs[j],ys[j]),
                       color='C{}'.format(j))
        axs[1].annotate(s='N={}'.format(int(N)), 
                       xy=(xs[j],ys2[j]),
                       color='C{}'.format(j))
#axs[0].legend([], [], loc='upper right', title='learned constraints', fontsize=10, framealpha=0.0)
axs[1].grid()
axs[0].grid()
savefig('plots/{}.pdf'.format(plotname), fig=fig)